In [2]:
import json

with open('weather-gov.json', 'r') as file:
    data = json.load(file)



In [3]:
print(data['features'][0]['properties']['sent'])
print(data['features'][0]['properties']['effective'])
print(data['features'][0]['properties']['onset'])
print(data['features'][0]['properties']['expires'])
print(data['features'][0]['properties']['messageType'])
print(data['features'][0]['properties']['severity'])
print(data['features'][0]['properties']['urgency'])
print(data['features'][0]['properties']['event'])
print(data['features'][0]['properties']['sender'])
print(data['features'][0]['properties']['senderName'])
print(data['features'][0]['properties']['headline'])
print(data['features'][0]['properties']['description'])
print(data['features'][0]['properties']['instruction'])
print(data['features'][0]['properties']['response'])

2024-11-04T09:55:00-06:00
2024-11-04T09:55:00-06:00
2024-11-04T09:55:00-06:00
2024-11-04T18:00:00-06:00
Alert
Moderate
Expected
Wind Advisory
w-nws.webmaster@noaa.gov
NWS Houston/Galveston TX
Wind Advisory issued November 4 at 9:55AM CST until November 4 at 6:00PM CST by NWS Houston/Galveston TX
* WHAT...South winds 15 to 25 mph with gusts up to 35 mph.

* WHERE...Portions of south central and southeast Texas.

* WHEN...Until 6 PM CST this evening.

* IMPACTS...Gusty winds will blow around unsecured objects. Tree
limbs could be blown down and a few power outages may result.
Winds this strong can make driving difficult, especially for high
profile vehicles. Use extra caution.
Execute


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import requests

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeatherAlerts") \
    .getOrCreate()

# Function to fetch weather alerts using Weather.gov API
def fetch_weather_alerts(lat, lon):
    try:
        url = f"https://api.weather.gov/alerts/active?point={lat},{lon}"
        headers = {"User-Agent": "MyWeatherApp (your-email@example.com)"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            alerts = data.get('features', [])
            return "; ".join([alert['properties']['headline'] for alert in alerts])
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return str(e)

# Register the function as a UDF
fetch_weather_alerts_udf = udf(fetch_weather_alerts, StringType())

# Example DataFrame with latitude and longitude
data = [
    {"latitude": 25.7617, "longitude": -80.1918},  # Miami, FL
    {"latitude": 34.0522, "longitude": -118.2437},  # Los Angeles, CA
]
df = spark.createDataFrame(data)

# Add a column with weather alerts
df_with_alerts = df.withColumn("alerts", fetch_weather_alerts_udf(col("latitude"), col("longitude")))

# Show results
df_with_alerts.show(truncate=False)

# Stop SparkSession
spark.stop()


your 131072x1 screen size is bogus. expect trouble
24/11/28 19:45:03 WARN Utils: Your hostname, LAPTOP-I1NR4E03 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/11/28 19:45:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 19:45:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+---------+------+
|latitude|longitude|alerts|
+--------+---------+------+
|25.7617 |-80.1918 |      |
|34.0522 |-118.2437|      |
+--------+---------+------+

